In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
from PIL import Image

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread

In [ ]:
import os
import zipfile

# Path to the zip file
zip_file_path = '/content/archive (3).zip'

# Directory where you want to extract the files
extracted_dir = '/content/extracted_data'

# Ensure the directory exists
os.makedirs(extracted_dir, exist_ok=True)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# List the contents of the extracted directory
extracted_files = os.listdir(extracted_dir)
print("Contents of the extracted directory:")
for file_name in extracted_files:
    print(file_name)

# Destination directory to save the extracted files
save_dir = '/content/saved_files'

# Ensure the directory exists
os.makedirs(save_dir, exist_ok=True)

# Move the extracted files to the destination directory
for file_name in extracted_files:
    src_path = os.path.join(extracted_dir, file_name)
    dst_path = os.path.join(save_dir, file_name)
    os.rename(src_path, dst_path)

print("Extracted files saved to:", save_dir)


In [ ]:

my_data_dir = '/content/drive/MyDrive/skin_cancer/data'

In [ ]:
os.listdir(my_data_dir)

In [ ]:
test_path = my_data_dir+'/test/'
train_path = my_data_dir+'/train/'

In [ ]:
os.listdir(test_path)

In [ ]:
os.listdir(train_path)

In [ ]:
os.listdir(train_path+'benign')[0]

In [ ]:
benign = train_path + 'benign/' + '1608.jpg'

In [ ]:
benign

In [ ]:
imread(benign)

In [ ]:
imread(benign).shape

In [ ]:
plt.imshow(imread(benign))

In [ ]:
os.listdir(train_path+'malignant')[0]

In [ ]:
malignant = train_path + 'malignant/' + '1381.jpg'

In [ ]:
malignant

In [ ]:
imread(malignant).shape

In [ ]:
plt.imshow(imread(malignant))

In [ ]:
len(os.listdir(train_path+'benign'))

In [ ]:
len(os.listdir(train_path+'malignant'))

In [ ]:
len(os.listdir(test_path+'benign'))

In [ ]:
len(os.listdir(test_path+'malignant'))

In [ ]:
imread(benign).shape

In [ ]:
imread(malignant).shape

In [ ]:
dim1 = []
dim2 = []

for image_filename in os.listdir(test_path+'malignant'):

    img = imread(test_path+'malignant/'+image_filename)
    d1,d2,colors = img.shape
    dim1.append(d1)
    dim2.append(d2)

In [ ]:
dim1

In [ ]:
sns.jointplot(dim1,dim2)

In [ ]:
np.mean(dim1)

In [ ]:
np.mean(dim2)

In [ ]:
image_shape = (224, 224, 3)

In [ ]:
imread(benign).max()

In [ ]:
imread(malignant).max()

#ImageDataGenerator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#for training images
train_datagen = ImageDataGenerator(rotation_range=20,
                               width_shift_range=0.10,
                               height_shift_range=0.10,
                               shear_range=0.1,
                               zoom_range=0.1,
                               horizontal_flip=True,
                               fill_mode='nearest',
                               rescale = 1./255
                              )

#for testing images
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
plt.imshow(imread(benign))

In [ ]:
plt.imshow(train_datagen.random_transform(imread(benign)))

#Creating the Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Dropout, Flatten, Dense, Conv2D, MaxPooling2D, BatchNormalization

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
optimizer = Adam(lr = 0.001)

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (3, 3), input_shape = image_shape, padding = 'Same', activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(filters = 32, kernel_size = (3, 3), padding = 'Same', activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.35))

model.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = 'Same',  activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.4))


model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy',
              optimizer = optimizer ,
              metrics = ['accuracy'])

In [ ]:
model.summary()

#Early Stopping

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [ ]:
early_stop = EarlyStopping(monitor = "val_accuracy", min_delta= 0.00005, patience= 7, verbose= 1, restore_best_weights= True)

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.5, patience = 5, min_lr = 1e-7,verbose=1)

#Training the Model

In [ ]:
batch_size = 32

In [ ]:
image_shape[:2]

In [ ]:
train_image_gen = train_datagen.flow_from_directory(train_path,
                                               target_size=image_shape[:2],
                                                color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary')

In [ ]:
test_image_gen = test_datagen.flow_from_directory(test_path,
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary',shuffle=False)

In [ ]:
train_image_gen.class_indices

In [ ]:
model.fit_generator(train_image_gen,epochs=100,
                              validation_data=test_image_gen,
                             callbacks=[early_stop, reduce_lr])

#Evaluating the Model

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses

In [ ]:
losses[['loss','val_loss']].plot()

In [ ]:
losses[['accuracy','val_accuracy']].plot()

In [ ]:
model.metrics_names

In [ ]:
model.evaluate_generator(test_image_gen)

In [ ]:
preds = model.predict_generator(test_image_gen)

In [ ]:
preds

In [ ]:
test_image_gen.classes

In [ ]:
# let's assume that > 0.5 belong to malignant and <= 0.5 belong to benign
predictions = preds > 0.5

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(test_image_gen.classes,predictions))

In [ ]:
confusion_matrix(test_image_gen.classes,predictions)

In [ ]:
from tensorflow.keras.models import load_model
model.save('/content/drive/MyDrive/skin_cancer_detection.h5')